In [2]:
import pandas as pd
import os
import random
from pprint import pprint
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from numpy import dot 
from numpy.linalg import norm
from pororo import Pororo
from transformers import AutoTokenizer

In [3]:
data = pd.read_csv("/opt/ml/code/generation/all_data.csv")
print(data.columns)

Index(['Unnamed: 0', 'index', 'title', 'context', 'hashtag', 'category'], dtype='object')


In [18]:
ner=Pororo(task='ner',lang="ko")
for i in range(20400,28169):
    word_tag=ner(data['hashtag'][i])
    for j in range(len(word_tag)):
        if word_tag[j][1]=='CITY' or word_tag[j][1]=='LOCATION':
            data['hashtag'][i]=data['hashtag'][i].replace(word_tag[j][0]," ")
            print("태그 삭제")
            print(i)
            print(word_tag[j][0]) 

태그 삭제
20446
대구
태그 삭제
20446
대구경북


/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


태그 삭제
20474
대구
태그 삭제
20528
시흥
태그 삭제
20528
수도권
태그 삭제
20528
인천
태그 삭제
20537
광주
태그 삭제
20624
인천
태그 삭제
20707
익산
태그 삭제
20742
사북
태그 삭제
20742
강원도
태그 삭제
20820
이수역
태그 삭제
20820
사당역
태그 삭제
20900
전주
태그 삭제
20993
부산
태그 삭제
21206
경산
태그 삭제
21206
대구
태그 삭제
21217
인천
태그 삭제
21217
부천
태그 삭제
21217
시흥
태그 삭제
21217
구로
태그 삭제
21234
울산
태그 삭제
21293
충주
태그 삭제
21318
포천
태그 삭제
21318
양주
태그 삭제
21320
의정부
태그 삭제
21456
구미
태그 삭제
21694
광주광역시
태그 삭제
21744
대전
태그 삭제
21996
창원
태그 삭제
22014
부산
태그 삭제
22166
대전
태그 삭제
22174
원주
태그 삭제
22371
경산
태그 삭제
22371
울산
태그 삭제
22488
대구
태그 삭제
23304
대전
태그 삭제
23406
주안역
태그 삭제
23878
청주
태그 삭제
23878
충주
태그 삭제
23878
대전
태그 삭제
23878
세종
태그 삭제
23878
원주
태그 삭제
23936
수원
태그 삭제
23941
수원
태그 삭제
23945
수원
태그 삭제
24004
인천
태그 삭제
24005
수원
태그 삭제
24008
안산
태그 삭제
24009
대전
태그 삭제
24011
대전
태그 삭제
24043
안산
태그 삭제
24074
안산
태그 삭제
24082
광주
태그 삭제
24084
천안
태그 삭제
24084
서울
태그 삭제
24084
군산
태그 삭제
24084
수원
태그 삭제
24085
천안
태그 삭제
24085
서울
태그 삭제
24085
군산
태그 삭제
24085
수원
태그 삭제
24146
울산
태그 삭제
24147
울산
태그 삭제
24148
울산
태그 삭제
24149
울산
태그 삭제
24151
울산
태그 삭제
24152
울산
태

KeyError: 28168

In [4]:
for i in range(len(data['hashtag'])):
    if type(data['hashtag'][i])==type(0.1):
        data=data.drop(i)

In [7]:
data=data.drop('level_0',axis=1)
data=data.drop(13798)
data=data.reset_index()

In [6]:
data=data.reset_index()

In [8]:
data

,level_0,Unnamed: 0,index,title,context,hashtag,category
0,0,0,0,에어팟프로 미개봉 (맥세이프호환),• 제품 - [애플] 에어팟 프로 (AirPods Pro) 2021년형 맥세이프...,"에어팟프로,에어팟프로미개봉,에어팟프로맥세이프,airpodspr,airpods",이어폰
1,1,1,3,버즈라이브 팝니다.,상태 좋아요 하자없어요.직거래 인천 입니다.,"버즈,버즈라이브",이어폰
2,2,2,5,에어팟2세대 유선충전 미개봉 (애플코리아정품),• 제품 - 에어팟2세대 유선충전• 상태 - 새제품 (미개봉)• 가격 - ...,"에어팟,에어팟미개봉,에어팟유선,에어팟2세대,무선에어팟",이어폰
3,3,3,7,@낱개판매전문@ 에어팟1/2세대 본체 왼쪽 오른쪽 유닛 팝니다,"- 저희 몬데몬데는 에어팟 낱개판매 '초창기 업자'이기 때문에 안좋은 후기가 있고,...","1세대,2세대,에어팟1,에어팟2,새상품",이어폰
4,4,4,8,갤럭시 버즈라이브 케이스,"원가 15,000원 주고 구매했던 버즈라이브 케이스 입니다.","버즈라이브,케이스,갤럭시,삼성전자,정품",이어폰
...,...,...,...,...,...,...,...
28164,28165,28904,35810,울산중고폰:갤럭시 탭A 10.1 32GB 그레이 팝니다 중고폰,울산중고폰 전문매장 중고폰마켓JJ입니다.갤럭시탭A(T515)10.1 32GB,"울산중고폰,부산중고폰,갤럭시탭,지패드,아이패드",테블릿
28165,28166,28905,35811,당구 디지털 점수대 태블릿 22인치,당구장사용태블릿입니다 당구점수대로도 사용가능하고 유튜브영상 볼수있습니다 남양주진접...,태블릿,테블릿
28166,28167,28906,35812,아이패드 프로 4세대 스파이스 그레이 128기가 11인치,"아이패드 프로 4세대구요11인치, 128기가 와이파이 입니다.** 아이패드 프로 4...","아이패드,아이패드프로,아이패드프로4세대",테블릿
28167,28168,28907,35813,아이패드 에어 4세대 새상품,아이패드 에어 4세대 새상품 입니다.,"아이패드에어4,아이패드새상품,256기가,와이파이,셀룰러",테블릿


In [27]:
for i in range(len(data['hashtag'])):
    if type(data['hashtag'][i])==type(0.1):


In [15]:
for i in range(20000, 20400 ):
    print(i)
    word_peice=ner(data['hashtag'][i])

20000
20001
20002
20003
20004


Process ForkProcess-86613:
Process ForkProcess-86610:
Exception ignored in: <function _releaseLock at 0x7f93b22610e0>
Traceback (most recent call last):
  File "/opt/conda/envs/lightweight/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Process ForkProcess-86611:
Process ForkProcess-86609:
Process ForkProcess-86614:
Traceback (most recent call last):
Process ForkProcess-86612:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/envs/lightweight

Process ForkProcess-86574:
Process ForkProcess-86575:
Process ForkProcess-86576:
Process ForkProcess-86570:
Process ForkProcess-86571:
Process ForkProcess-86572:
Process ForkProcess-86573:
Traceback (most recent call last):
Process ForkProcess-86569:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/lightweight/lib/pyth

Process ForkProcess-83351:
Process ForkProcess-83350:
Process ForkProcess-83352:
Process ForkProcess-83349:
Process ForkProcess-83348:
Process ForkProcess-83347:
Process ForkProcess-83346:
Process ForkProcess-83345:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/envs/lightweight/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/envs/lightwei

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [13]:
data=data.drop(20400)

In [12]:
ner(data['hashtag'][20400])

IndexError: list index out of range

In [14]:
data=data.drop('level_0',axis=1)
data=data.reset_index()

In [19]:
data.to_csv('hashtag_complete.csv')

In [25]:
count=0
for i in range(20000,28000):
    
    print(data['hashtag'][i])
        

40인치 UHD LED TV ( 모니터,무료배송 ),40인치 UHD,새제품
49인치 UHD,셋톱박스,모니터,uhd,50인치
삼성패널,55인치,티비
인터넷,삼성,캠핑,엘지,현금지원
리모콘,lg리모콘
티비,tv,32인치,40인치
고장난tv,모니터,lg,패널,메인보드
LGTV,LG모니터,와이드탁상용TV,26인치TV,24인치TV
사운드바,사운드바추천,삼성사운드바,사운드바새것
영업용 냉장냉동고
TV,65인치
lg,LGTV49인치,49인치티비,49인치TV,엘지
티비모니터,티비,lg,lgtv
tv,스마트티비,티비,스마트라tv,화질굿
UHDTV,75인치,루컴즈,75인치tv,TV렌탈
LG티비,엘쥐티비32인치,lg32인치led,lg32인치티비
led,led간판,led전광판,led조명
50인치tv,TV,티비,LGTV,TV받침
스마트tv,65인치,삼성tv
와이드스텐드,LED스텐드,전등,LED전등
LED간판
엘지스탠바이미,스탠바이미,lg스탠바이미,스탠바이미미개봉,스탠바이미lg
거울,붙이는거울,DIY,욕실거울,전신거울
엘지,Xcanvas,TV,리모컨
tv,스탠드
삼성,TV스텐드,삼성TV
삼성,파브,pdp,TV
Tv,스텐드,삼성
LG,LGTV,TV,HDTV,LG울트라HDTV
렌탈,QLED,TV,65인치,삼성가전
렌탈,UHD,TV,50인치,삼성가전
40인치 LED UHD TV(모니터,무료배송),40인치 LED,셋톱박스
65인치 UHD 4K LED TV 새제품 무료배송 모니터,티비,텔레비전,지상파,셋톱박스
엘지티비,49인치uhd
Tv,티비,삼성,엘지,LCD
삼성24인치tv,삼성tv,삼성24인치,캠핑용tv,T24H315
tv다이,티비다이,3만,원목, 
텔레비젼
엘지65인치,중고티비,65인치,리퍼티비,중소기업티비
엘지티비,65인치,중고티비,리퍼티비,중소기업티비
엘지티비,리퍼티비,김포티비
브라켓,벽걸이
엘피,LP
TV,LGTV,모니터,42인치,벽걸이TV
티비,65인치,TV,텔레비전
LG,티비,TV,레트로티비
led,전기등,30w,led방등
삼성,TV,32인치,LEDTV,32
미사용,새상품,정품,

In [22]:
print(count)

7993


In [23]:
data

,level_0,Unnamed: 0,index,title,context,hashtag,category
0,0,0,0,에어팟프로 미개봉 (맥세이프호환),• 제품 - [애플] 에어팟 프로 (AirPods Pro) 2021년형 맥세이프...,"에어팟프로,에어팟프로미개봉,에어팟프로맥세이프,airpodspr,airpods",이어폰
1,1,1,3,버즈라이브 팝니다.,상태 좋아요 하자없어요.직거래 인천 입니다.,"버즈,버즈라이브",이어폰
2,2,2,5,에어팟2세대 유선충전 미개봉 (애플코리아정품),• 제품 - 에어팟2세대 유선충전• 상태 - 새제품 (미개봉)• 가격 - ...,"에어팟,에어팟미개봉,에어팟유선,에어팟2세대,무선에어팟",이어폰
3,3,3,7,@낱개판매전문@ 에어팟1/2세대 본체 왼쪽 오른쪽 유닛 팝니다,"- 저희 몬데몬데는 에어팟 낱개판매 '초창기 업자'이기 때문에 안좋은 후기가 있고,...","1세대,2세대,에어팟1,에어팟2,새상품",이어폰
4,4,4,8,갤럭시 버즈라이브 케이스,"원가 15,000원 주고 구매했던 버즈라이브 케이스 입니다.","버즈라이브,케이스,갤럭시,삼성전자,정품",이어폰
...,...,...,...,...,...,...,...
28163,28164,28904,35810,울산중고폰:갤럭시 탭A 10.1 32GB 그레이 팝니다 중고폰,울산중고폰 전문매장 중고폰마켓JJ입니다.갤럭시탭A(T515)10.1 32GB,"중고폰, 중고폰,갤럭시탭,지패드,아이패드",테블릿
28164,28165,28905,35811,당구 디지털 점수대 태블릿 22인치,당구장사용태블릿입니다 당구점수대로도 사용가능하고 유튜브영상 볼수있습니다 남양주진접...,태블릿,테블릿
28165,28166,28906,35812,아이패드 프로 4세대 스파이스 그레이 128기가 11인치,"아이패드 프로 4세대구요11인치, 128기가 와이파이 입니다.** 아이패드 프로 4...","아이패드,아이패드프로,아이패드프로4세대",테블릿
28166,28167,28907,35813,아이패드 에어 4세대 새상품,아이패드 에어 4세대 새상품 입니다.,"아이패드에어4,아이패드새상품,256기가,와이파이,셀룰러",테블릿


In [26]:
data1=pd.read_csv('./hashtag20400.csv')

In [27]:
for i in range(10000,20000):
    print(data1['hashtag'][i])

와인,와인냉장고,와인셀러,12병,매직쉐프
한샘,오젠,건조기,한샘오젠,한샘오젠건조기
냉장고,중고냉장고,냉장쇼케이스,음료수냉장고
냉장고,중고냉장고,일반냉장고
세탁기,중고세탁기,일반세탁기,통세탁기
냉장고,중고냉장고,양문형냉장고
냉장고,중고냉장고,양문형냉장고
냉장고,중고냉장고,양문형냉장고,매직스페이스
냉장고,중고냉장고,양문형냉장고
냉장고,중고냉장고,일반냉장고
세탁기,중고세탁기,드럼세탁기,소형세탁기
세탁기,중고세탁기,드럼세탁기
중랑구중고가전,광진구중고가전,하남중고가전,의정부중고가전,남양주중고가전
하남중고가전,강동구중고가전,의정부중고가전,광진구중고가전,중랑구중고가전
간냉식,냉장고
 중고가전, 중고가전, 중고가전, 중고가전, 중고가전
하남중고가전,중랑구중고가전,광진구중고가전,강동구중고가전,의정부중고가전
세탁기,드럼세탁기
냉장고,중고냉장고,일반냉장고,소형냉장고
세탁기,중고세탁기,엘지세탁기,통돌이세탁기
세탁기,일반세탁기
세탁기,중고세탁기,일반세탁기,통세탁기,중고가전
에어컨,중고에어컨,냉난방기,중고냉난방기
세탁기,일반세탁기,통세탁기
35평,40평,50평,냉난방기,캐리어
쇼케이스,케이스,커피케이스,커피
냉장고,김치냉장고,위니아,딤채김치냉장고
미니냉장고,냉장고,위니아,냉장냉동
4도어냉장고
렌탈,세탁기,건조기,트롬,LG가전
그랜드우성,테이블냉장고150,5자,보냉테이블
유니크,업소용냉장고,2냉동1냉장,새제품
냉동고,카페,우성,서랍식테이블냉장고
냉난방기
냉장고,냉동고,투도어,김치냉장고,삼성엘지
테이블,냉장고
공기청정기,급처
삼성1도어김치냉장,김치냉장고,냉장고
삼성그랑데세탁기건조기,비스포크세탁기,비스포크건조기,삼성세탁기,삼성건조기
삼성김치냉장고,비스포크김치냉장고,김치냉장고3도어,삼성김치냉장고3도
삼성김치냉장고,비스포크김치냉장고,김치냉장고1도어,김치냉장고,1도어김치냉장고
비스포크냉장고,삼성비스포크냉장고,삼성냉장고,스탠드냉장고,냉장고
제빙기,제빙기110kg,제빙기110키로
렌탈,세탁기렌탈,엘지세탁기,통돌이세탁기
냉장고렌탈,세탁기렌탈,렌탈,삼성냉장고,삼성건조기
렌탈,티비렌탈,세탁기렌탈,루컴즈


In [29]:
print(ner('중랑구중고가전'))

[('중랑구중고가전', 'EVENT')]


In [45]:
data1=pd.read_csv('./hashtag10000_complete.csv')
data2=pd.read_csv('./hashtag20400.csv')
data3=pd.read_csv('./hashtag_complete.csv')

In [46]:
list=[x for x in range(10000,28170)]

In [47]:
data1=data1.drop(index=list,axis=0)

In [57]:
list2=[x for x in range(10000)]
list3=[x for x in range(20400,28168)]


In [60]:
data2=data2.drop(index=list3,axis=0)

In [61]:
data2

,Unnamed: 0,level_0,Unnamed: 0.1,index,title,context,hashtag,category
10000,10000,10000,10233,12393,매직쉐프 와인셀러 (새상품 무료배송),매직쉐프 와인 냉장고미개봉 새상품입니다 무료배송입니다,"와인,와인냉장고,와인셀러,12병,매직쉐프",대형가전
10001,10001,10001,10234,12394,한샘 오젠 건조기 5kg (새상품),한샘 오젠 건조기5kg,"한샘,오젠,건조기,한샘오젠,한샘오젠건조기",대형가전
10002,10002,10002,10235,12395,냉장 쇼케이스,"냉장 쇼케이스 , 2018년식.","냉장고,중고냉장고,냉장쇼케이스,음료수냉장고",대형가전
10003,10003,10003,10236,12396,삼성 335리터 냉장고,"삼성 냉장고335리터 , 2017년식.","냉장고,중고냉장고,일반냉장고",대형가전
10004,10004,10004,10237,12397,삼성 액티브워시 21키로 세탁기,"삼성 액티브워시 세탁기21키로 , 2017년식.사다리 작업시 사다리비용 추가됩니다....","세탁기,중고세탁기,일반세탁기,통세탁기",대형가전
...,...,...,...,...,...,...,...,...
20395,20395,20395,21032,26087,청주 대전 삼성 엘지LED디지털tv팝니다,삼성 엘지 LED LCD PDP 디지털TV 판매합니다.전제품 삼성 엘지 중고 제품으...,,tv
20396,20396,20396,21033,26088,인터넷 TV 신규가입 이벤트중입니다!!,번개장터을 통해 인터넷TV 신규 가입시 현금 60만원까지 지원해드리고 있습니다문자&...,"인터넷,TV,이벤트",tv
20397,20397,20397,21034,26090,85인치 삼성 스마트 TV UHD 4K 미사용 리퍼제품 (안심구매),*삼성 스마트 TV 85인치 2020년형 *미사용 리퍼제품 UHD 4K ...,"85인치TV,85TU8000,스마트TV,리퍼tv,중고티비",tv
20398,20398,20398,21035,26091,75인치 삼성 스마트 TV UHD 4K 미사용 리퍼제품 (안심구매),*삼성 스마트 TV 75인치 2018년형 *미사용 리퍼제품 UHD 4K ...,"75인치TV,75NU6900,리퍼tv,스마트TV,중고티비",tv


In [67]:
list3=[x for x in range(20400)  ]

In [66]:
data3=pd.read_csv('./hashtag_complete.csv')

In [68]:
data3=data3.drop(index=list3,axis=0)

In [69]:
data3

,Unnamed: 0,level_0,Unnamed: 0.1,index,title,context,hashtag,category
20400,20400,20401,21038,26094,70인치 삼성 스마트 TV UHD 4K 미사용 리퍼제품 (안심구매),*삼성 스마트 TV 70인치 2020년형 *미사용 리퍼제품 UHD 4K ...,"70인치TV,70TU7000,스마트TV,리퍼tv,중고티비",tv
20401,20401,20402,21039,26095,55인치 삼성 스마트 TV UHD 4K 미사용 리퍼제품 (안심구매),*삼성 스마트 TV 55인치 2018년형 *미사용 리퍼제품 UHD 4K ...,"55인치TV,55NU6900,스마트TV,리퍼tv,중고티비",tv
20402,20402,20403,21040,26096,55인치 삼성 스마트 TV UHD 4K 미사용 리퍼제품 (안심구매),*삼성 스마트 TV 55인치 2019년형 *미사용 리퍼제품 UHD 4K ...,"55인치TV,55RU7300,스마트TV,리퍼tv,중고티비",tv
20403,20403,20404,21041,26097,인터넷 티비 신규 이벤트,당근마켓을 통해 인터넷 TV가입시 현금지급 60만원까지 제공해 드리고 있습니다!😍인...,"인터넷가입,TV신규가입,현금지원,필름무료,삼성휴대폰",tv
20404,20404,20405,21042,26098,인터넷 티비 신규 가입 이벤트,당근마켓을 통해 인터넷 TV가입시 현금지급 60만원까지 제공해 드리고 있습니다!😍인...,"인터넷가입,TV신규가입,현금지원,필름무료,삼성휴대폰",tv
...,...,...,...,...,...,...,...,...
28163,28163,28164,28904,35810,울산중고폰:갤럭시 탭A 10.1 32GB 그레이 팝니다 중고폰,울산중고폰 전문매장 중고폰마켓JJ입니다.갤럭시탭A(T515)10.1 32GB,"중고폰, 중고폰,갤럭시탭,지패드,아이패드",테블릿
28164,28164,28165,28905,35811,당구 디지털 점수대 태블릿 22인치,당구장사용태블릿입니다 당구점수대로도 사용가능하고 유튜브영상 볼수있습니다 남양주진접...,태블릿,테블릿
28165,28165,28166,28906,35812,아이패드 프로 4세대 스파이스 그레이 128기가 11인치,"아이패드 프로 4세대구요11인치, 128기가 와이파이 입니다.** 아이패드 프로 4...","아이패드,아이패드프로,아이패드프로4세대",테블릿
28166,28166,28167,28907,35813,아이패드 에어 4세대 새상품,아이패드 에어 4세대 새상품 입니다.,"아이패드에어4,아이패드새상품,256기가,와이파이,셀룰러",테블릿


In [70]:
result=pd.concat([data1,data2,data3])

In [73]:
result1=result.drop(['Unnamed: 0'],axis=1)

In [76]:
result1=result.drop(['level_0','Unnamed: 0.1'],axis=1)

In [80]:
result1=result1.reset_index()

In [82]:
result1.to_csv('hashtag_processing_complete.csv')

In [ ]:
result1.